In [28]:
import pandas as pd
import nltk
import pickle

In [29]:

data=pd.read_csv("/home/dalmas/Documents/sentiment analytical system/twitter-airline-sentiment/Tweets.csv")

In [30]:
sentiment=data[["airline_sentiment","text"]]

In [31]:
sentiment.head()

,airline_sentiment,text
0,neutral,@VirginAmerica What @dhepburn said.
1,positive,@VirginAmerica plus you've added commercials t...
2,neutral,@VirginAmerica I didn't today... Must mean I n...
3,negative,@VirginAmerica it's really aggressive to blast...
4,negative,@VirginAmerica and it's a really big bad thing...


In [32]:
sentiment.shape

(14640, 2)

In [33]:
words=[]
for statement in sentiment["text"]:
    words.append([word for word in statement.split()  if not word.startswith('@')])

In [34]:
from nltk.corpus import stopwords

In [35]:
stop_words = set(stopwords.words('english')) 

In [36]:
filtered_sentences=[]
for sentence in words:
    filtered_sentences.append([w.split('.')[0].lower() for w in sentence if not w in stop_words])

In [37]:
all_words=[]
for fs in filtered_sentences:
    all_words+=fs

In [38]:
frequency_dist = nltk.FreqDist(all_words)

In [39]:
common=frequency_dist.most_common(2500)
features = [word for word,_ in common  if not word.startswith('fl')]
word_features= [word for word in features  if not word.startswith('http')]

In [40]:
cols=pd.DataFrame(word_features).T

In [41]:
cols.to_csv("ml_app/features.csv")

In [42]:
def find_features(document):
    words = set(document)
    features = {}
    for w in word_features:
        features[w]=(w in words)
    return features

In [43]:
featureset=[]
for i in range(sentiment.shape[0]):
    featureset.append((find_features(filtered_sentences[i]),sentiment["airline_sentiment"][i]))

In [44]:
featureset[1]

({'i': False,
  'get': False,
  'cancelled': False,
  'thanks': False,
  'service': False,
  'customer': False,
  "i'm": False,
  'time': False,
  '-': False,
  '2': False,
  '&amp;': False,
  'us': False,
  'help': False,
  'hours': False,
  'hold': False,
  'thank': False,
  'you': False,
  'still': False,
  'need': False,
  'plane': False,
  'would': False,
  'one': False,
  "can't": False,
  'please': False,
  'gate': False,
  'call': False,
  'back': False,
  'delayed': False,
  'got': False,
  'hour': False,
  'late': False,
  'like': False,
  'bag': False,
  'phone': False,
  'no': False,
  'airline': False,
  'waiting': False,
  'trying': False,
  'know': False,
  'guys': False,
  'it': False,
  'u': False,
  'going': False,
  '': False,
  'way': False,
  'today': False,
  'never': False,
  '3': False,
  "i've": False,
  'make': False,
  'last': False,
  'great': False,
  'now': False,
  'wait': False,
  'airport': False,
  'change': False,
  'even': False,
  'good': False,
  '

In [45]:
s=int(2*sentiment.shape[0]/3)
training_set = featureset[:s]
testing_set = featureset[s:]

In [46]:
from nltk.classify.scikitlearn import SklearnClassifier
from sklearn.naive_bayes import BernoulliNB

In [47]:
BNB_classifier = SklearnClassifier(BernoulliNB())
BNB_classifier.train(training_set)
print("BNB_classifier alg acc: ",(nltk.classify.accuracy(BNB_classifier,testing_set))*100)

BNB_classifier alg acc:  79.77459016393442


In [48]:
BNB_classifier.classify(training_set[1][0])

'positive'

In [49]:
training_set[1][1]

'positive'

In [50]:
import pickle

In [51]:
with open('model.pickle', 'wb') as f:
    pickle.dump(BNB_classifier, f, pickle.HIGHEST_PROTOCOL)

In [52]:
with open('model.pickle', 'rb') as f:
    ml_model = pickle.load(f)

In [53]:
ml_model.classify(training_set[1][0])

'positive'

In [54]:
'dali'.split('.')

['dali']